In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xml.etree.cElementTree as ET
import glob
import os
import json
import random
import shutil

from PIL import Image, ImageOps

###Review Data

In [ ]:
# This is a label from the annotation folder. This label is .xml format.
with open("/content/drive/MyDrive/AIFFEL/AIFFELTON/label_data/co_dataset/xml_file/xml_file/128_1b3_Tc_mc_LittC2SE_0.xml") as f:
    contents = f.read()
    print(contents)

<?xml version="1.0" ?>
<annotation>
    <folder>crackles_image</folder>
    <filename>128_1b3_Tc_mc_LittC2SE_0_s.png</filename>
    <path>/content/drive/MyDrive/AIFFEL/AIFFELTON/label_data/crackles_image/128_1b3_Tc_mc_LittC2SE_0_crackles.png</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
        <width>969</width>
        <height>370</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>crackles</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>261</xmin>
            <ymin>258</ymin>
            <xmax>705</xmax>
            <ymax>352</ymax>
        </bndbox>
    </object>
</annotation>



###Preparing Data For Model(YoloV9)

####Translate '.xml" format to ".txt" format

In [ ]:
# The functions are about translating label data.
def xml_to_yolo_bbox(bbox, w, h):
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2 ) / h

    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h

    return [x_center, y_center, width, height]

def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center, width, height
    w_half_len = (bbox[2] + w) / 2
    h_half_len = (bbox[3] + h) / 2

    xmin = int((bbox[0] + w) - w_half_len)
    ymin = int((bbox[1] + h) - h_half_len)
    xmax = int((bbox[0] + w) + w_half_len)
    ymax = int((bbox[1] + h) + h_half_len)

    return [xmin, ymin, xmax, ymax]

In [ ]:
# Create .txt label data.
classes = []

input_dir = "/kaggle/input/face-mask-detection/annotations"
output_dir = "/kaggle/working/labels"
image_dir = "/kaggle/input/face-mask-detection/images"

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

files = glob.glob(os.path.join(input_dir, "*.xml"))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist")
        continue

    result = []

    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall("object"):
        label = obj.find("name").text

        if label not in classes:
            classes.append(label)

        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)

        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:

        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding = "utf-8") as f:
            f.write("\n".join(result))

with open(f"{output_dir}/classes.txt", "w", encoding = "utf-8") as f:
    f.write(json.dumps(classes))

In [ ]:
# Labels
with open(f"{output_dir}/classes.txt") as f:
    contents = f.read()
    print(contents)

In [ ]:
# .xml format
with open("/kaggle/input/face-mask-detection/annotations/maksssksksss334.xml") as f:
    contents = f.read()
    print(contents)

In [ ]:
# .txt format
with open("/kaggle/working/labels/maksssksksss334.txt") as f:
    contents = f.read()
    print(contents)

In [ ]:
annotation_count = len(os.listdir("/content/drive/MyDrive/AIFFEL/AIFFELTON/label_data/co_dataset/xml_file/xml_file"))
labels_count = len(os.listdir("/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV7/labels"))
print(f"Annotation Counts: {annotation_count}")
print(f"Lables Counts: {labels_count}")

Annotation Counts: 6898
Lables Counts: 6898


###Preparing Training Data

In [4]:
# Create a folder of train dataset
if not os.path.isfile("/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data"):
    # os.mkdir('/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data')
    os.mkdir('/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/train')
    os.mkdir('/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/val')
    os.mkdir('/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/test')
    os.mkdir('/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/train/images')
    os.mkdir('/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/train/labels')
    os.mkdir('/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/test/images')
    os.mkdir('/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/test/labels')
    os.mkdir('/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/val/images')
    os.mkdir('/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/val/labels')

In [5]:
# Material is the names of images in the images folder.
metarial = []
for i in os.listdir("/content/drive/MyDrive/AIFFEL/AIFFELTON/label_data/co_dataset/mel_image"):
    str = i[:-13]
    metarial.append(str)

In [6]:
metarial[0:10]

['192_2b3_Ar_mc_LittC2SE_1',
 '192_2b3_Ar_mc_LittC2SE_2',
 '192_2b3_Ar_mc_LittC2SE_3',
 '192_2b3_Ar_mc_LittC2SE_4',
 '192_2b3_Ar_mc_LittC2SE_5',
 '192_2b3_Ar_mc_LittC2SE_6',
 '193_1b2_Al_mc_AKGC417L_0',
 '193_1b2_Al_mc_AKGC417L_1',
 '193_1b2_Al_mc_AKGC417L_2',
 '193_1b2_Al_mc_AKGC417L_3']

In [7]:
print("Sum of image: ", len(metarial))
train_size = int(len(metarial) * 0.7)
test_size = int(len(metarial) * 0.15)
val_size = int(len(metarial) * 0.15)
print("Sum of the train size: ", train_size)
print("Sum of the test size: ", test_size)
print("Sum of the val size: ", val_size)

Sum of image:  6898
Sum of the train size:  4828
Sum of the test size:  1034
Sum of the val size:  1034


In [8]:
import os
import shutil

def preparingdata(main_txt_file, main_img_file, train_size, test_size, val_size):
    for i in range(0, train_size):
        source_txt = main_txt_file + "/" + metarial[i] + ".txt"
        source_img = main_img_file + "/" + metarial[i] + "_mel_spec.png"

        mstring = metarial[i]
        train_destination_txt = "/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/train/labels" + "/" + metarial[i] + ".txt"
        train_destination_png = "/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/train/images" + "/" + metarial[i] + "_mel_spec.png"

        shutil.copy(source_txt, train_destination_txt)
        shutil.copy(source_img, train_destination_png)

    for l in range(train_size , train_size + test_size):
        source_txt = main_txt_file + "/" + metarial[l] + ".txt"
        source_img = main_img_file + "/" + metarial[l] + "_mel_spec.png"

        mstring = metarial[l]
        test_destination_txt = "/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/test/labels" + "/" + metarial[l] + ".txt"
        test_destination_png = "/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/test/images" + "/" + metarial[l] + "_mel_spec.png"

        shutil.copy(source_txt, test_destination_txt)
        shutil.copy(source_img, test_destination_png)

    for n in range(train_size + test_size , train_size + test_size + val_size):
        source_txt = main_txt_file + "/" + metarial[n] + ".txt"
        source_img = main_img_file + "/" + metarial[n] + "_mel_spec.png"

        mstring = metarial[n]
        val_destination_txt = "/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/val/labels" + "/" + metarial[n] + ".txt"
        val_destination_png = "/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/val/images" + "/" + metarial[n] + "_mel_spec.png"

        # Remove _mel_spec from the filename
        new_filename = metarial[n].replace("_mel_spec", "")

        # Copy files with modified filename
        shutil.copy(source_txt, val_destination_txt)
        shutil.copy(source_img, val_destination_png.replace(metarial[n] + "_mel_spec", new_filename))


In [9]:
#preparinbdata("/kaggle/working/labels", "/kaggle/input/face-mask-detection/images", 603, 150, 100)
preparingdata(main_txt_file = "/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV7/labels",
              main_img_file = "/content/drive/MyDrive/AIFFEL/AIFFELTON/label_data/co_dataset/mel_image",
              train_size = 4828,
              test_size = 1034,
              val_size = 1034)

In [12]:
train_data = len(os.listdir("/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/train/images"))
vol_data = len(os.listdir("/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/val/images"))
test_data = len(os.listdir("/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/test/images"))

print(f"train_data: {train_data}")
print(f"vol_data: {vol_data}")
print(f"test_data: {test_data}")

train_data: 4828
vol_data: 1034
test_data: 1034


###yaml 파일 제작

In [19]:
# Create the data.yaml. I am going to use it on the train with yolov9.
yaml_text = """train: /content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/train/images/
val: /content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/val/images

nc: 3
names: ["Normal", "crackles", "wheezes"]"""

with open("/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/data.yaml", 'w') as file:
    file.write(yaml_text)

In [20]:
with open("/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/data.yaml") as f:
    contents = f.read()
    print(contents)

train: /content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/train/images/
val: /content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/Training/data/val/images

nc: 3
names: ["Normal", "crackles", "wheezes"]


###YoloV9

In [1]:
# Download Github rep.
!git clone https://github.com/SkalskiP/yolov9.git

Cloning into 'yolov9'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 325 (delta 134), reused 102 (delta 102), pack-reused 153
Receiving objects: 100% (325/325), 2.26 MiB | 1.09 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [2]:
# İnstall the req.
!pip install -r /content/yolov9/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [3]:
# Train with YoloV9
"""
workers --> max dataloader workers (per RANK in DDP mode)
device --> cuda device, i.e. 0 or 0,1,2,3 or cpu
batch --> total batch size for all GPUs, -1 for autobatch
epochs --> total training epochs
data --> dataset.yaml path
img --> train, val image size (pixels)
cfg --> model.yaml path
weights --> initial weights path
name --> save to project/name
hyp --> hyperparameters path

**This parameters is my project parameters. You can make changes for your project.**
"""

!python /content/yolov9/train_dual.py \
--workers 8 --device cpu --batch 16 --epochs 50 --min-items 0 --close-mosaic 15\
--data /content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/data.yaml \
--img 640 --cfg /content/yolov9/models/detect/yolov9-c.yaml \
--weights '' --name yolov9-c --hyp /content/yolov9/data/hyps/hyp.scratch-high.yaml

2024-03-15 09:06:29.468402: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 09:06:29.468493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 09:06:29.470846: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-15 09:06:30.769818: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train_dual: weights=, cfg=/content/yolov9/models/detect/yolov9-c.yaml, data=/content/drive/MyDrive/AIFFEL/AIFFELTON/model/YoloV9/data.yaml, hyp=/content/yolov9/data/hyps/hyp.scratch-high.yaml, epochs=50, batch_size=16, imgsz=640, rect=False,